# Linear Regression - Credit Rating Prediction

Simple linear regression to predict sovereign credit ratings from macroeconomic indicators.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## 1. Load Data

In [ ]:
df = pd.read_csv('../data/processed/merged_dataset.csv')
print(f'Dataset shape: {df.shape}')
df.head()

## 2. Prepare X and y

In [ ]:
X = df.drop(['Country', 'Year', 'Credit_Rating'], axis=1)
y = df['Credit_Rating']

print(f'Features: {list(X.columns)}')
print(f'X shape: {X.shape}, y shape: {y.shape}')

## 3. Split Train/Test (80/20)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Train set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')

## 4. Train Linear Regression Model

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
print('✓ Model trained')

## 5. Make Predictions

In [ ]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)
print('✓ Predictions made')

## 6. Evaluate Model

In [ ]:
# Training metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_r2 = r2_score(y_train, y_train_pred)

print('Training Set:')
print(f'  RMSE: {train_rmse:.4f}')
print(f'  MAE:  {train_mae:.4f}')
print(f'  R²:   {train_r2:.4f}\n')

# Test metrics
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_r2 = r2_score(y_test, y_test_pred)

print('Test Set:')
print(f'  RMSE: {test_rmse:.4f}')
print(f'  MAE:  {test_mae:.4f}')
print(f'  R²:   {test_r2:.4f}')

## 7. Feature Importance (Coefficients)

In [ ]:
coefficients_df = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
}).sort_values('Coefficient', key=abs, ascending=False)

print('Feature Importance:')
print(coefficients_df)

## 8. Visualization

## 9. K-Fold Cross-Validation (K=5)

In [ ]:
# K-fold cross-validation with K=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Calculate scores for each metric
rmse_scores = -cross_val_score(model, X, y, cv=kfold, scoring='neg_root_mean_squared_error')
mae_scores = -cross_val_score(model, X, y, cv=kfold, scoring='neg_mean_absolute_error')
r2_scores = cross_val_score(model, X, y, cv=kfold, scoring='r2')

# Print results for each fold
print('Results per fold:')
for i in range(5):
    print(f'  Fold {i+1}: RMSE={rmse_scores[i]:.4f}, MAE={mae_scores[i]:.4f}, R²={r2_scores[i]:.4f}')

# Calculate mean and std
print(f'\nAverage across 5 folds:')
print(f'  RMSE: {rmse_scores.mean():.4f} ± {rmse_scores.std():.4f}')
print(f'  MAE:  {mae_scores.mean():.4f} ± {mae_scores.std():.4f}')
print(f'  R²:   {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')

## 10. Comparison: Simple Split vs K-Fold CV

In [ ]:
# Create comparison table
comparison_df = pd.DataFrame({
    'Method': ['Simple Split (80/20)', 'K-Fold CV (K=5)'],
    'RMSE': [test_rmse, rmse_scores.mean()],
    'MAE': [test_mae, mae_scores.mean()],
    'R²': [test_r2, r2_scores.mean()],
    'Std': ['-', f'±{r2_scores.std():.4f}']
})

print('Comparison of Methods:')
print(comparison_df.to_string(index=False))

print('\nConclusion:')
print('- Simple split gave R² = 0.4668 (optimistic, best fold)')
print('- K-fold CV gives R² = 0.4395 ± 0.016 (more realistic average)')
print('- True performance is likely around R² ≈ 0.44')

## 11. K-Fold CV Visualization

In [ ]:
# Visualize K-fold CV results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: R² scores per fold
folds = [f'Fold {i+1}' for i in range(5)]
ax1.bar(folds, r2_scores, alpha=0.7, color='steelblue', edgecolor='black')
ax1.axhline(y=r2_scores.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {r2_scores.mean():.4f}')
ax1.axhline(y=test_r2, color='green', linestyle='--', linewidth=2, label=f'Simple Split: {test_r2:.4f}')
ax1.set_ylabel('R² Score', fontsize=12)
ax1.set_title('R² Score per Fold (K-Fold CV)', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Plot 2: Box plot of all metrics
metrics_data = [rmse_scores, mae_scores, r2_scores]
ax2.boxplot(metrics_data, labels=['RMSE', 'MAE', 'R²'])
ax2.set_ylabel('Score', fontsize=12)
ax2.set_title('Distribution of Metrics (K-Fold CV)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print('\nVisualization shows:')
print('- Left: R² varies between folds (0.42 to 0.47)')
print('- Right: Distribution of all metrics across folds')
print('- Red line: K-fold average')
print('- Green line: Simple split result (best fold)')

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Actual vs Predicted
ax1.scatter(y_test, y_test_pred, alpha=0.6, edgecolors='k')
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
ax1.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2)
ax1.set_xlabel('Actual Credit Rating', fontsize=12)
ax1.set_ylabel('Predicted Credit Rating', fontsize=12)
ax1.set_title('Actual vs Predicted', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Plot 2: Feature Importance
colors = ['green' if x < 0 else 'red' for x in coefficients_df['Coefficient']]
ax2.barh(coefficients_df['Feature'], coefficients_df['Coefficient'], color=colors, alpha=0.7)
ax2.set_xlabel('Coefficient Value', fontsize=12)
ax2.set_title('Feature Importance (Coefficients)', fontsize=14, fontweight='bold')
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax2.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## 12. Polynomial Regression (Degree 2)

In [ ]:
# Create polynomial features (degree 2)
poly2 = PolynomialFeatures(degree=2, include_bias=False)
X_poly2 = poly2.fit_transform(X)

print(f'Original features: {X.shape[1]}')
print(f'Polynomial features (degree=2): {X_poly2.shape[1]}')

# K-fold cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
model_poly2 = LinearRegression()

rmse_poly2 = -cross_val_score(model_poly2, X_poly2, y, cv=kfold, scoring='neg_root_mean_squared_error')
mae_poly2 = -cross_val_score(model_poly2, X_poly2, y, cv=kfold, scoring='neg_mean_absolute_error')
r2_poly2 = cross_val_score(model_poly2, X_poly2, y, cv=kfold, scoring='r2')

print('\nResults per fold:')
for i in range(5):
    print(f'  Fold {i+1}: RMSE={rmse_poly2[i]:.4f}, MAE={mae_poly2[i]:.4f}, R²={r2_poly2[i]:.4f}')

print(f'\nAverage across 5 folds:')
print(f'  RMSE: {rmse_poly2.mean():.4f} ± {rmse_poly2.std():.4f}')
print(f'  MAE:  {mae_poly2.mean():.4f} ± {mae_poly2.std():.4f}')
print(f'  R²:   {r2_poly2.mean():.4f} ± {r2_poly2.std():.4f}')

## 13. Polynomial Regression (Degree 3)

In [ ]:
# Create polynomial features (degree 3)
poly3 = PolynomialFeatures(degree=3, include_bias=False)
X_poly3 = poly3.fit_transform(X)

print(f'Original features: {X.shape[1]}')
print(f'Polynomial features (degree=3): {X_poly3.shape[1]}')

# K-fold cross-validation
model_poly3 = LinearRegression()

rmse_poly3 = -cross_val_score(model_poly3, X_poly3, y, cv=kfold, scoring='neg_root_mean_squared_error')
mae_poly3 = -cross_val_score(model_poly3, X_poly3, y, cv=kfold, scoring='neg_mean_absolute_error')
r2_poly3 = cross_val_score(model_poly3, X_poly3, y, cv=kfold, scoring='r2')

print('\nResults per fold:')
for i in range(5):
    print(f'  Fold {i+1}: RMSE={rmse_poly3[i]:.4f}, MAE={mae_poly3[i]:.4f}, R²={r2_poly3[i]:.4f}')

print(f'\nAverage across 5 folds:')
print(f'  RMSE: {rmse_poly3.mean():.4f} ± {rmse_poly3.std():.4f}')
print(f'  MAE:  {mae_poly3.mean():.4f} ± {mae_poly3.std():.4f}')
print(f'  R²:   {r2_poly3.mean():.4f} ± {r2_poly3.std():.4f}')

## 14. Comparison: Linear vs Polynomial Regression

In [ ]:
# Create comparison table
comparison_models = pd.DataFrame({
    'Model': ['Linear (K-Fold)', 'Polynomial deg=2', 'Polynomial deg=3'],
    'Mean_R²': [r2_scores.mean(), r2_poly2.mean(), r2_poly3.mean()],
    'Std_R²': [r2_scores.std(), r2_poly2.std(), r2_poly3.std()],
    'Mean_RMSE': [rmse_scores.mean(), rmse_poly2.mean(), rmse_poly3.mean()],
    'Mean_MAE': [mae_scores.mean(), mae_poly2.mean(), mae_poly3.mean()],
    'Features': [X.shape[1], X_poly2.shape[1], X_poly3.shape[1]]
})

print('Model Comparison:')
print(comparison_models.to_string(index=False))

print('\n' + '='*70)
print('ANALYSIS:')
print('='*70)
print(f'1. Linear Regression:')
print(f'   - R² = {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')
print(f'   - Stable baseline (low std)')
print(f'\n2. Polynomial deg=2:')
print(f'   - R² = {r2_poly2.mean():.4f} ± {r2_poly2.std():.4f}')
print(f'   - Best R² (+{(r2_poly2.mean()-r2_scores.mean())*100:.1f}% improvement)')
print(f'   - BUT: High variability (std 13x larger than Linear)')
print(f'   - Conclusion: Better performance but unstable → Overfitting risk')
print(f'\n3. Polynomial deg=3:')
print(f'   - R² = {r2_poly3.mean():.4f} ± {r2_poly3.std():.4f}')
print(f'   - NEGATIVE R² → Worse than predicting the mean!')
print(f'   - 164 features for 950 observations → Severe overfitting')
print(f'   - Conclusion: Unusable model')
print('\n' + '='*70)
print('WINNER: Polynomial deg=2 (best R²) but needs regularization to stabilize')
print('='*70)

## 15. Polynomial Regression Visualization

In [ ]:
# Visualize Polynomial Regression results
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))

# Plot 1: Linear Regression R² per fold
folds = [f'Fold {i+1}' for i in range(5)]
ax1.bar(folds, r2_scores, alpha=0.7, color='steelblue', edgecolor='black')
ax1.axhline(y=r2_scores.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {r2_scores.mean():.4f}')
ax1.set_ylabel('R² Score', fontsize=12)
ax1.set_title('Linear Regression\nR² Score per Fold', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim([-0.1, 0.8])

# Plot 2: Polynomial deg=2 R² per fold
ax2.bar(folds, r2_poly2, alpha=0.7, color='coral', edgecolor='black')
ax2.axhline(y=r2_poly2.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {r2_poly2.mean():.4f}')
ax2.set_ylabel('R² Score', fontsize=12)
ax2.set_title('Polynomial deg=2\nR² Score per Fold', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3, axis='y')
ax2.set_ylim([-0.1, 0.8])

# Plot 3: Polynomial deg=3 R² per fold
ax3.bar(folds, r2_poly3, alpha=0.7, color='darkred', edgecolor='black')
ax3.axhline(y=r2_poly3.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {r2_poly3.mean():.4f}')
ax3.set_ylabel('R² Score', fontsize=12)
ax3.set_title('Polynomial deg=3\nR² Score per Fold (OVERFITTING)', fontsize=14, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3, axis='y')
ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)

plt.tight_layout()
plt.show()

print('\nVisualization Analysis:')
print('- Linear: Stable performance across all folds (R² ≈ 0.42-0.47)')
print('- Poly deg=2: Better average but high variability (R² = 0.12 to 0.70)')
print('- Poly deg=3: Catastrophic - negative R² on most folds (overfitting)')

## 16. Ridge Regression (Linear Features)

In [ ]:
# Test Ridge Regression with different alpha values
alphas = [0.01, 0.1, 1.0, 10.0]

# Normalize features (REQUIRED for Ridge!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print('Testing Ridge Regression with different alpha values...\n')

ridge_results = {}
for alpha in alphas:
    model_ridge = Ridge(alpha=alpha)
    
    # K-fold CV
    rmse_ridge = -cross_val_score(model_ridge, X_scaled, y, cv=kfold, scoring='neg_root_mean_squared_error')
    mae_ridge = -cross_val_score(model_ridge, X_scaled, y, cv=kfold, scoring='neg_mean_absolute_error')
    r2_ridge = cross_val_score(model_ridge, X_scaled, y, cv=kfold, scoring='r2')
    
    ridge_results[alpha] = {
        'rmse': rmse_ridge,
        'mae': mae_ridge,
        'r2': r2_ridge
    }
    
    print(f'Alpha = {alpha:5.2f}:')
    print(f'  RMSE: {rmse_ridge.mean():.4f} ± {rmse_ridge.std():.4f}')
    print(f'  MAE:  {mae_ridge.mean():.4f} ± {mae_ridge.std():.4f}')
    print(f'  R²:   {r2_ridge.mean():.4f} ± {r2_ridge.std():.4f}\n')

## 17. Ridge Regression Comparison

In [ ]:
# Create comparison table
ridge_comparison = pd.DataFrame({
    'Alpha': alphas,
    'Mean_R²': [ridge_results[a]['r2'].mean() for a in alphas],
    'Std_R²': [ridge_results[a]['r2'].std() for a in alphas],
    'Mean_RMSE': [ridge_results[a]['rmse'].mean() for a in alphas],
    'Mean_MAE': [ridge_results[a]['mae'].mean() for a in alphas]
})

print('Ridge Regression Comparison:')
print(ridge_comparison.to_string(index=False))

print('\n' + '='*70)
print('ANALYSIS:')
print('='*70)
print(f'Linear (K-Fold) baseline: R² = {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')
print('\nRidge Regression results:')
for alpha in alphas:
    r2_mean = ridge_results[alpha]['r2'].mean()
    r2_std = ridge_results[alpha]['r2'].std()
    diff = (r2_mean - r2_scores.mean()) * 100
    print(f'  Alpha = {alpha:5.2f}: R² = {r2_mean:.4f} ± {r2_std:.4f} ({diff:+.2f}%)')

print('\n' + '='*70)
print('CONCLUSION:')
print('Ridge does NOT improve Linear Regression on 8 features')
print('All alpha values give essentially the same performance as Linear')
print('Reason: Linear is already stable (low variance) with only 8 features')
print('Ridge regularization is not needed for this simple model')
print('='*70)

## 18. Ridge Regression Visualization

In [ ]:
# Visualize Ridge Regression results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: R² vs Alpha
mean_r2_ridge = [ridge_results[a]['r2'].mean() for a in alphas]
std_r2_ridge = [ridge_results[a]['r2'].std() for a in alphas]

ax1.plot(alphas, mean_r2_ridge, 'o-', linewidth=2, markersize=8, color='darkblue', label='Ridge')
ax1.axhline(y=r2_scores.mean(), color='red', linestyle='--', linewidth=2, 
            label=f'Linear baseline: {r2_scores.mean():.4f}')
ax1.fill_between(alphas, 
                  [m - s for m, s in zip(mean_r2_ridge, std_r2_ridge)],
                  [m + s for m, s in zip(mean_r2_ridge, std_r2_ridge)],
                  alpha=0.2, color='darkblue')
ax1.set_xlabel('Alpha (λ)', fontsize=12)
ax1.set_ylabel('R² Score', fontsize=12)
ax1.set_title('Ridge Regression: R² vs Regularization Strength', fontsize=14, fontweight='bold')
ax1.set_xscale('log')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Stability (Std) vs Alpha
ax2.plot(alphas, std_r2_ridge, 'o-', linewidth=2, markersize=8, color='darkgreen', label='Ridge Std')
ax2.axhline(y=r2_scores.std(), color='red', linestyle='--', linewidth=2, 
            label=f'Linear Std: {r2_scores.std():.4f}')
ax2.set_xlabel('Alpha (λ)', fontsize=12)
ax2.set_ylabel('Standard Deviation of R²', fontsize=12)
ax2.set_title('Ridge Regression: Stability vs Regularization Strength', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print('\nVisualization Analysis:')
print('- Left: Ridge R² is identical to Linear for all alpha values')
print('- Right: Stability (std) is the same as Linear')
print('- Conclusion: Ridge regularization has no effect on 8-feature Linear model')

## 19. Ridge + Polynomial Regression (Degree 2)

In [ ]:
# Test Ridge + Polynomial with different alpha values
alphas = [0.01, 0.1, 1.0, 10.0]

# Create polynomial features
poly_rp = PolynomialFeatures(degree=2, include_bias=False)
X_poly_rp = poly_rp.fit_transform(X)

# Normalize polynomial features (REQUIRED for Ridge!)
scaler_rp = StandardScaler()
X_poly_rp_scaled = scaler_rp.fit_transform(X_poly_rp)

print(f'Original features: {X.shape[1]}')
print(f'Polynomial features (degree=2): {X_poly_rp.shape[1]}')
print(f'Features normalized for Ridge regularization\n')

ridge_poly_results = {}
for alpha in alphas:
    model_rp = Ridge(alpha=alpha)
    
    # K-fold CV
    rmse_rp = -cross_val_score(model_rp, X_poly_rp_scaled, y, cv=kfold, scoring='neg_root_mean_squared_error')
    mae_rp = -cross_val_score(model_rp, X_poly_rp_scaled, y, cv=kfold, scoring='neg_mean_absolute_error')
    r2_rp = cross_val_score(model_rp, X_poly_rp_scaled, y, cv=kfold, scoring='r2')
    
    ridge_poly_results[alpha] = {
        'rmse': rmse_rp,
        'mae': mae_rp,
        'r2': r2_rp
    }
    
    print(f'Alpha = {alpha:5.2f}:')
    print(f'  RMSE: {rmse_rp.mean():.4f} ± {rmse_rp.std():.4f}')
    print(f'  MAE:  {mae_rp.mean():.4f} ± {mae_rp.std():.4f}')
    print(f'  R²:   {r2_rp.mean():.4f} ± {r2_rp.std():.4f}\n')

## 20. Final Model Comparison

In [ ]:
# Create final comparison table
final_comparison = pd.DataFrame({
    'Model': [
        'Linear (K-Fold)',
        'Polynomial deg=2',
        'Ridge + Poly α=0.01',
        'Ridge + Poly α=0.1',
        'Ridge + Poly α=1.0',
        'Ridge + Poly α=10.0'
    ],
    'Mean_R²': [
        r2_scores.mean(),
        r2_poly2.mean(),
        ridge_poly_results[0.01]['r2'].mean(),
        ridge_poly_results[0.1]['r2'].mean(),
        ridge_poly_results[1.0]['r2'].mean(),
        ridge_poly_results[10.0]['r2'].mean()
    ],
    'Std_R²': [
        r2_scores.std(),
        r2_poly2.std(),
        ridge_poly_results[0.01]['r2'].std(),
        ridge_poly_results[0.1]['r2'].std(),
        ridge_poly_results[1.0]['r2'].std(),
        ridge_poly_results[10.0]['r2'].std()
    ],
    'Mean_RMSE': [
        rmse_scores.mean(),
        rmse_poly2.mean(),
        ridge_poly_results[0.01]['rmse'].mean(),
        ridge_poly_results[0.1]['rmse'].mean(),
        ridge_poly_results[1.0]['rmse'].mean(),
        ridge_poly_results[10.0]['rmse'].mean()
    ]
})

print('='*70)
print('FINAL MODEL COMPARISON')
print('='*70)
print(final_comparison.to_string(index=False))

print('\n' + '='*70)
print('ANALYSIS:')
print('='*70)
print(f'1. Linear (baseline): R² = {r2_scores.mean():.4f} ± {r2_scores.std():.4f}')
print(f'   - Stable but limited performance')
print(f'\n2. Polynomial deg=2: R² = {r2_poly2.mean():.4f} ± {r2_poly2.std():.4f}')
print(f'   - Better R² (+23%) but VERY unstable (Std 13x larger)')
print(f'\n3. Ridge + Polynomial α=10.0: R² = {ridge_poly_results[10.0]["r2"].mean():.4f} ± {ridge_poly_results[10.0]["r2"].std():.4f}')
print(f'   - BEST MODEL: +47% vs Linear, 4x more stable than Poly alone')
print(f'   - Regularization (α=10) successfully stabilizes the model')

print('\n' + '='*70)
print('WINNER: Ridge + Polynomial (α=10.0)')
print('  - R² = 0.6477 → Explains 65% of credit rating variance')
print('  - Stable (Std = 0.057)')
print('  - Best compromise between performance and reliability')
print('='*70)

## 21. Final Visualization: Model Comparison

In [ ]:
# Final visualization comparing all key models
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Data for visualization
models = ['Linear', 'Poly deg=2', 'Ridge+Poly\nα=0.01', 'Ridge+Poly\nα=0.1', 
          'Ridge+Poly\nα=1.0', 'Ridge+Poly\nα=10']
r2_means_final = [
    r2_scores.mean(),
    r2_poly2.mean(),
    ridge_poly_results[0.01]['r2'].mean(),
    ridge_poly_results[0.1]['r2'].mean(),
    ridge_poly_results[1.0]['r2'].mean(),
    ridge_poly_results[10.0]['r2'].mean()
]
r2_stds_final = [
    r2_scores.std(),
    r2_poly2.std(),
    ridge_poly_results[0.01]['r2'].std(),
    ridge_poly_results[0.1]['r2'].std(),
    ridge_poly_results[1.0]['r2'].std(),
    ridge_poly_results[10.0]['r2'].std()
]

colors = ['steelblue', 'coral', 'lightgreen', 'lightgreen', 'lightgreen', 'gold']
x_pos = np.arange(len(models))

# Plot 1: R² comparison with error bars
bars = ax1.bar(x_pos, r2_means_final, yerr=r2_stds_final, alpha=0.7, color=colors, 
               edgecolor='black', capsize=5, linewidth=1.5)
ax1.set_ylabel('R² Score', fontsize=12)
ax1.set_title('Model Comparison: R² Performance', fontsize=14, fontweight='bold')
ax1.set_xticks(x_pos)
ax1.set_xticklabels(models, rotation=45, ha='right')
ax1.grid(True, alpha=0.3, axis='y')
ax1.axhline(y=r2_scores.mean(), color='red', linestyle='--', linewidth=1, 
            alpha=0.5, label='Linear baseline')

# Add value labels on bars
for i, (bar, mean, std) in enumerate(zip(bars, r2_means_final, r2_stds_final)):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + std + 0.01,
             f'{mean:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

ax1.legend()
ax1.set_ylim([0, 0.75])

# Plot 2: Stability comparison
ax2.bar(x_pos, r2_stds_final, alpha=0.7, color=colors, edgecolor='black', linewidth=1.5)
ax2.set_ylabel('Standard Deviation of R²', fontsize=12)
ax2.set_title('Model Comparison: Stability (Lower is Better)', fontsize=14, fontweight='bold')
ax2.set_xticks(x_pos)
ax2.set_xticklabels(models, rotation=45, ha='right')
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (x, std) in enumerate(zip(x_pos, r2_stds_final)):
    ax2.text(x, std + 0.005, f'{std:.3f}', ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print('\n' + '='*70)
print('VISUALIZATION INSIGHTS:')
print('='*70)
print('LEFT PLOT (Performance):')
print('  - Ridge+Poly α=10 (gold) has highest R² = 0.648')
print('  - Progressive improvement with stronger regularization')
print('  - Error bars show confidence intervals (±1 std)')
print('\nRIGHT PLOT (Stability):')
print('  - Poly deg=2 (coral) is very unstable (Std = 0.215)')
print('  - Ridge+Poly α=10 (gold) is 4x more stable (Std = 0.057)')
print('  - Regularization successfully reduces variance')
print('='*70)